In [23]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
import copy

In [2]:
night=Image.open('vangogh-starry-night.png')
w, h = night.size
reduced=np.array(night)/255
vecs=reduced.reshape(7644,5,5).reshape(7644,25)

In [53]:
class KmeanVQ:
    def __init__(self,data,init='rand',C=3):
        self.data=data
        self.init=init
        self.C=C
        self.labels=np.zeros(len(data))
        self.colors=['r','g','b','cyan','magenta','purple','k']
        self.Cov=np.cov(self.data)
        if self.init=='rand':
            self.Random()
            self.mean()
        elif self.init=='forgy':
            self.Forgy()
            self.cluster()
        elif self.init=='K++':
            self.Kpp()
            self.cluster()
        else:
            print('Initialization type not recognized defaulting to random')
            self.init='rand'
            self.Random()
            self.mean()
    def Random(self):
            self.mu=[[-1,-1]]*self.C
            y=np.random.randint(0,self.C,len(self.labels))
            test=1
            for i in range(self.C):
                test*=len(np.where(y==i)[0])
            if test==0:
                while test==0:
                    y=np.random.randint(0,self.C,len(self.labels))
                    test=1
                    for i in range(self.C):
                        test*=len(np.where(y==i)[0])
            self.labels=y
    def Forgy(self):
        self.mu=[]
        sample=[]
        while len(np.unique(sample))!=self.C:
            sample=np.random.randint(0,len(self.data),self.C)
        for a in sample:
            muk=list(self.data[a].values())
            self.mu.append(muk)       
    def Kpp(self):
        self.mu=[]
        self.mu.append(self.data[np.random.randint(0,len(self.data))])
        while len(self.mu)!=self.C:
            prob=np.zeros(len(self.data))
            for i in range(len(self.data)):
                t=[0]*len(self.mu)
                for k in range(len(self.mu)):
                    t[k]=self.Dist2(self.data[i],self.mu[k])**2
                prob[i]=min(t)
            prob/=sum(prob)
            s=np.random.choice(np.arange(0,len(prob),1),p=prob)
            self.mu.append(self.data[s])
    def Dist2(self,x,y):
        d=np.sum((x-y)**2)
        return d
    def cluster(self):
        for i in range(len(self.data)):
            t=[0]*self.C
            for k in range(self.C):
                t[k]=self.Dist2(self.data[i],self.mu[k])
            self.labels[i]=np.argmin(t)
    def mean(self):
        for c in range(self.C):
            z=np.where(self.labels==c)
            muk=[np.mean(self.data[z],axis=0)]
            self.mu[c]=muk
    def Loss(self,norm=False):
        E=0
        for c in range(self.C):
            z=np.where(self.labels==c)
            for i in z[0]:
                E+=self.Dist2(self.data[i],self.mu[c])
        if norm:
            return E/np.trace(self.Cov)
        else:
            return(E)
    def run(self,itt=100):
        for i in range(itt):
            self.oldmu = copy.deepcopy(self.mu)
            self.mean()
            self.cluster()
            #self.plot()
            if np.array_equal(self.oldmu,self.mu):
                print('Convergence Reached',i,' Itterations')
                break
            if i ==itt:
                print('Max Itteration Reached')

In [ ]:
K=KmeanVQ(vecs,'K++',C=1000)
K.run()

In [ ]:
comp=[]
for i in range(len(vecs)):
    comp.append(K.mu[int(K.labels[i])])

In [ ]:
img=np.array(comp).reshape(7644,5,5).reshape(h,w)

In [ ]:
plt.imshow(img)

In [47]:
np.sum((K.mu[0]-K.mu[1])**2)

1.705359477124183

In [22]:
list.stream

AttributeError: type object 'list' has no attribute 'stream'